# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime
import json
from shapely.geometry import shape
import geopandas as gpd
import geojson

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
279578,39520568,44,2,50,1,1032018,C50911,1,49,8,4,10,1.0
451457,34958209,36,2,36,1,24072016,D0511,1,66,8,23,3,1.0
259037,14342823,13,2,13,1,19082016,C50412,1,79,4,27,3,1.0
459950,19288210,23,2,23,2,3022017,C9101,1,19,1,29,30,1.0
14177,39730498,45,1,45,2,24032017,A080,1,0,1,18,5,1.0


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title,resumen_cap
51,F64,Trastornos de identidad de género,F60-69,Trastornos de la personalidad y del comportami...,T. de la personalidad/comportamiento
10,F10,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...,consumo psicotrópicos
75,F98,Otros trastornos emocionales y de comportamien...,F90-F98,Trastornos emocionales y del comportamiento qu...,aparecen en niñez


In [12]:
diagnos.code.unique()

array(['F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17',
       'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F40', 'F41',
       'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53',
       'F54', 'F55', 'F59', 'F60', 'F62', 'F63', 'F64', 'F66', 'F68',
       'F69', 'F65', 'F70', 'F71', 'F72', 'F73', 'F78', 'F79', 'F80',
       'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 'F90', 'F91', 'F92',
       'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [13]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [14]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [15]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [16]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
752236,48,1,48,2,25052018,F321,1,50,14,1.00000,F32,F30-39
769613,2,2,2,2,15092017,F418,1,35,3,1.00000,F41,F40-49
740683,8,1,8,2,29072017,F29,1,40,1,1.00974,F29,F20-29


In [17]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

In [18]:
dicc_res = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_res[r.cap] = r.resumen_cap

In [19]:
dicc_res

{'F00-F09': 'orgánicos/sintomáticos',
 'F10-F19': 'consumo psicotrópicos',
 'F20-29': 'esquizotípicos/delirantes',
 'F30-39': 'animo/afectivos',
 'F40-49': 'nuróticos/somatomorfos',
 'F50-59': 'factores físicos /fisiológicos',
 'F60-69': 'T. de la personalidad/comportamiento',
 'F70-79': 'deficiencia psíquica',
 'F80-89': 'desarrollo psicológico',
 'F90-F98': 'aparecen en niñez',
 'F99': 'sin espcificar'}

In [20]:
morbilidad['resumen'] = morbilidad.cap.map(dicc_res)

In [21]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,resumen
753139,48,1,48,2,11012017,F3164,1,63,120,1.00000,F31,F30-39,animo/afectivos
760003,8,2,8,2,26102017,F329,1,40,1,1.00974,F32,F30-39,animo/afectivos
762748,48,1,48,1,15072016,F609,1,46,18,1.00000,F60,F60-69,T. de la personalidad/comportamiento


## Creamos fecha de ingreso

In [22]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [23]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [24]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [25]:
morbilidad['year']=morbilidad.fecha_ing.dt.year
morbilidad['month']=morbilidad.fecha_ing.dt.month
morbilidad['day']=morbilidad.fecha_ing.dt.day

In [26]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [27]:
# Cambiamos algunos datos categóricos que vienen como números 

In [28]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,resumen,fecha_al,fecha_ing,year,month,day
714751,33,1,33,2,F200,1,29,10,1.0,F20,F20-29,esquizotípicos/delirantes,2018-08-13,2018-08-03,2018,8,3
788283,20,2,20,2,F913,1,16,5,1.0,F91,F90-F98,aparecen en niñez,2018-01-05,2017-12-31,2017,12,31
745117,49,1,49,2,F22,1,51,14,1.0,F22,F20-29,esquizotípicos/delirantes,2019-12-05,2019-11-21,2019,11,21


In [29]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [30]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [31]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [32]:
#preparar provincias para geojsaon (nombre)

In [33]:
#diccionario
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [34]:
morbilidad['ProvHosp_nom'] = morbilidad.ProvHosp.map(cod_prov)
morbilidad['ProvResi_nom'] = morbilidad.ProvResi.map(cod_prov)

In [35]:
#prov resi 53 = extranjeros
morbilidad.ProvResi_nom.fillna('Extranjeros',inplace = True)

In [36]:
morbilidad.isna().sum()

ProvHosp        0
Sexo            0
ProvResi        0
DiagEntr        0
DiagPrin        0
MotivoAlta      0
EdadAnios       0
DiasEstancia    0
Factor          0
code            0
cap             0
resumen         0
fecha_al        0
fecha_ing       0
year            0
month           0
day             0
Entrada         0
Alta            0
SexNom          0
ProvHosp_nom    0
ProvResi_nom    0
dtype: int64

In [37]:
morbilidad.columns

Index(['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'DiagPrin', 'MotivoAlta',
       'EdadAnios', 'DiasEstancia', 'Factor', 'code', 'cap', 'resumen',
       'fecha_al', 'fecha_ing', 'year', 'month', 'day', 'Entrada', 'Alta',
       'SexNom', 'ProvHosp_nom', 'ProvResi_nom'],
      dtype='object')

In [38]:
morbilidad.rename({'EdadAnios':'Edad','DiasEstancia':'Dias'},axis=1,inplace = True)

In [39]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
734591,43,2,43,1,F329,1,17,24,1.00000,F32,...,2016-03-07,2016-02-12,2016,2,12,Ordinario,Curación,Mujer,Tarragona,Tarragona
768328,48,1,48,2,F919,1,42,8,1.00441,F91,...,2016-03-23,2016-03-15,2016,3,15,Urgente,Curación,Hombre,Vizcaya,Vizcaya
748085,07,2,07,2,F312,1,40,10,1.00000,F31,...,2017-01-13,2017-01-03,2017,1,3,Urgente,Curación,Mujer,Baleares,Baleares


In [40]:
morbilidad.to_csv('../data/morbilidad_Fgen.csv',index=False)

In [41]:
## extras

In [ ]:
dicc_poly = {pf['properties']['name']:pf['geometry'] for pf in provincias['features']}

In [ ]:
morbilidad['geometry'] = morbilidad.ProvHosp_nom.map(dicc_poly)

In [ ]:
morbilidad.sample(3)

In [ ]:
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"],
                                                    X["elev"])),
                            properties=dict(name=X["name"],
                                            description=X["description"])))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)